In [2]:
import baostock as bs
import pandas as pd
from datetime import datetime

In [3]:
# 登录系统
lg = bs.login()
print('login success!')
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

login success!
login success!
login respond error_code:0
login respond  error_msg:success


In [4]:
stock_code = "sh.600150"  # 要查询的股票代码

In [5]:
# 获取最近的数据
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = '2024-01-01'  # 获取最近2-3年的数据

print(f"正在下载 {stock_code} 从 {start_date} 到 {end_date} 的数据...")

正在下载 sh.600150 从 2024-01-01 到 2025-08-09 的数据...


In [6]:
# 查询日K线数据
rs = bs.query_history_k_data_plus(stock_code,
    "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST",
    start_date=start_date, 
    end_date=end_date,
    frequency="d", adjustflag="3")

print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success


In [7]:
# 打印结果集
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())

# 转换为DataFrame
result = pd.DataFrame(data_list, columns=rs.fields)

print(f"\n原始数据行数: {len(result)}")
print("\n查看前5行原始数据:")
print(result.head())



原始数据行数: 388

查看前5行原始数据:
         date       code     open     high      low    close preclose  \
0  2024-01-02  sh.600150  29.6300  30.5000  29.4300  29.8000  29.4400   
1  2024-01-03  sh.600150  29.6200  30.3000  29.6200  30.2500  29.8000   
2  2024-01-04  sh.600150  30.1300  30.2400  29.3900  29.8000  30.2500   
3  2024-01-05  sh.600150  29.6600  30.0900  29.2900  29.5200  29.8000   
4  2024-01-08  sh.600150  29.7600  29.8500  29.3100  29.4500  29.5200   

     volume           amount adjustflag      turn tradestatus     pctChg isST  
0  55697597  1672796799.6500          3  1.245400           1   1.222800    0  
1  42374787  1272760165.2800          3  0.947500           1   1.510100    0  
2  38794005  1152872630.9600          3  0.867400           1  -1.487600    0  
3  31292074   928770692.8600          3  0.699700           1  -0.939600    0  
4  29398328   868718073.4700          3  0.657300           1  -0.237100    0  


In [8]:
# 数据清洗和类型转换
# 先替换空字符串为NaN
result = result.replace('', pd.NA)

# 转换数据类型，使用 pd.to_numeric 来处理错误
result['date'] = pd.to_datetime(result['date'])
result['close'] = pd.to_numeric(result['close'], errors='coerce')
result['open'] = pd.to_numeric(result['open'], errors='coerce')
result['high'] = pd.to_numeric(result['high'], errors='coerce')
result['low'] = pd.to_numeric(result['low'], errors='coerce')
result['volume'] = pd.to_numeric(result['volume'], errors='coerce')
result['amount'] = pd.to_numeric(result['amount'], errors='coerce')
result['pctChg'] = pd.to_numeric(result['pctChg'], errors='coerce')
result['turn'] = pd.to_numeric(result['turn'], errors='coerce')

# 删除包含空值的行（特别是价格数据）
print(f"\n转换前数据行数: {len(result)}")
result = result.dropna(subset=['date', 'close', 'open', 'high', 'low'])
print(f"删除空值后数据行数: {len(result)}")

# 按日期排序
result = result.sort_values('date')
result = result.reset_index(drop=True)


转换前数据行数: 388
删除空值后数据行数: 388


In [9]:
# 保存为Excel文件(可选)
output_file = f'{stock_code}.xlsx'
result.to_excel(output_file, index=False)
print(f"\n数据已保存到: {output_file}")
print(f"共 {len(result)} 条有效记录")


数据已保存到: sh.600150.xlsx
共 388 条有效记录


In [10]:
# 登出系统
bs.logout()
print("\n登出成功！")

logout success!

登出成功！
